In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

# Load the data with embeddings (already segmented)
embeddings = pd.read_pickle(r"data folder\data\AT_with_embeddings_final.pkl")

print(f"✅ Loaded data: {embeddings.shape}")
print(f"Columns: {list(embeddings.columns)}")

✅ Loaded data: (231752, 30)
Columns: ['Sitting_ID', 'Speech_ID', 'Title', 'Date', 'Body', 'Term', 'Session', 'Meeting', 'Sitting', 'Agenda', 'Subcorpus', 'Lang', 'Speaker_role', 'Speaker_MP', 'Speaker_minister', 'Speaker_party', 'Speaker_party_name', 'Party_status', 'Party_orientation', 'Speaker_ID', 'Speaker_name', 'Speaker_gender', 'Speaker_birth', 'Text', 'Word_Count', 'Is_Too_Short', 'Is_Filtered', 'Speech_Embeddings', 'Segment_ID', 'Segment_Embeddings']


In [2]:
# === DATA OVERVIEW ===
print("📊 Data Overview:")
print(f"  • Total speeches: {embeddings.shape[0]:,}")
print(f"  • Speech embedding shape: {embeddings['Speech_Embeddings'][0].shape}")
print(f"  • Segment embedding shape: {embeddings['Segment_Embeddings'][0].shape}")
print(f"  • Unique segments: {embeddings['Segment_ID'].nunique():,}")
print(f"  • Average speeches per segment: {embeddings.shape[0] / embeddings['Segment_ID'].nunique():.1f}")

# Check for missing values
print(f"\n🔍 Missing values:")
print(f"  • Segment_ID: {embeddings['Segment_ID'].isna().sum()}")
print(f"  • Speech_Embeddings: {embeddings['Speech_Embeddings'].isna().sum()}")
print(f"  • Segment_Embeddings: {embeddings['Segment_Embeddings'].isna().sum()}")

# Check sitting length distribution
sitting_lengths = embeddings.groupby('Sitting_ID').size()
print(f"\n📈 Sitting length distribution:")
print(f"  • Min speeches per sitting: {sitting_lengths.min()}")
print(f"  • Max speeches per sitting: {sitting_lengths.max()}")
print(f"  • Average speeches per sitting: {sitting_lengths.mean():.1f}")
print(f"  • Sittings with <50 speeches: {(sitting_lengths < 50).sum()}")
print(f"  • Sittings with >200 speeches: {(sitting_lengths > 200).sum()}")

📊 Data Overview:
  • Total speeches: 231,752
  • Speech embedding shape: (1024,)
  • Segment embedding shape: (1024,)
  • Unique segments: 5,728
  • Average speeches per segment: 40.5

🔍 Missing values:
  • Segment_ID: 0
  • Speech_Embeddings: 41119
  • Segment_Embeddings: 0

📈 Sitting length distribution:
  • Min speeches per sitting: 1
  • Max speeches per sitting: 1378
  • Average speeches per sitting: 189.8
  • Sittings with <50 speeches: 474
  • Sittings with >200 speeches: 542
  • Segment_Embeddings: 0

📈 Sitting length distribution:
  • Min speeches per sitting: 1
  • Max speeches per sitting: 1378
  • Average speeches per sitting: 189.8
  • Sittings with <50 speeches: 474
  • Sittings with >200 speeches: 542


## Topic Modeling with BERTopic

This notebook focuses on topic modeling using pre-segmented parliamentary speeches. The data has already been processed with:
- Speech-level embeddings for similarity analysis
- Segment-level embeddings for topic modeling
- Parliamentary-aware segmentation with agenda detection

We'll implement hierarchical BERTopic with LLM classification to map topics to 22 policy categories.

In [ ]:
# === BERTOPIC SETUP WITH GUIDED TOPICS ===
from bertopic import BERTopic
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from umap import UMAP
import openai
import os
from dotenv import load_dotenv

# Define the 22 target topic categories
label_list = [
    "Education", "Technology", "Health", "Environment", "Housing", "Labor", 
    "Defense", "Government Operations", "Social Welfare", "Other", "Macroeconomics", 
    "Domestic Commerce", "Civil Rights", "International Affairs", "Transportation", 
    "Immigration", "Law and Crime", "Agriculture", "Foreign Trade", "Culture", 
    "Public Lands", "Energy"
]

# Detailed topic descriptions for better classification
majortopics_description = {
    'Macroeconomics': 'issues related to domestic macroeconomic policy, such as the state and prospect of the national economy, economic policy, inflation, interest rates, monetary policy, cost of living, unemployment rate, national budget, public debt, price control, tax enforcement, industrial revitalization and growth.',
    'Civil Rights': 'issues related to civil rights and minority rights, discrimination towards races, gender, sexual orientation, handicap, and other minorities, voting rights, freedom of speech, religious freedoms, privacy rights, protection of personal data, abortion rights, anti-government activity groups (e.g., local insurgency groups), religion and the Church.',
    'Health': 'issues related to health care, health care reforms, health insurance, drug industry, medical facilities, medical workers, disease prevention, treatment, and health promotion, drug and alcohol abuse, mental health, research in medicine, medical liability and unfair medical practices.',
    'Agriculture': 'issues related to agriculture policy, fishing, agricultural foreign trade, food marketing, subsidies to farmers, food inspection and safety, animal and crop disease, pest control and pesticide regulation, welfare for animals in farms, pets, veterinary medicine, agricultural research.',
    'Labor': 'issues related to labor, employment, employment programs, employee benefits, pensions and retirement accounts, minimum wage, labor law, job training, labor unions, worker safety and protection, youth employment and seasonal workers.',
    'Education': 'issues related to educational policies, primary and secondary schools, student loans and education finance, the regulation of colleges and universities, school reforms, teachers, vocational training, evening schools, safety in schools, efforts to improve educational standards, and issues related to libraries, dictionaries, teaching material, research in education.',
    'Environment': 'issues related to environmental policy, drinking water safety, all kinds of pollution (air, noise, soil), waste disposal, recycling, climate change, outdoor environmental hazards (e.g., asbestos), species and forest protection, marine and freshwater environment, hunting, regulation of laboratory or performance animals, land and water resource conservation, research in environmental technology.',
    'Energy': 'issues related to energy policy, electricity, regulation of electrical utilities, nuclear energy and disposal of nuclear waste, natural gas and oil, drilling, oil spills, oil and gas prices, heat supply, shortages and gasoline regulation, coal production, alternative and renewable energy, energy conservation and energy efficiency, energy research.',
    'Immigration': 'issues related to immigration, refugees, and citizenship, integration issues, regulation of residence permits, asylum applications; criminal offences and diseases caused by immigration.',
    'Transportation': 'issues related to mass transportation construction and regulation, bus transport, regulation related to motor vehicles, road construction, maintenance and safety, parking facilities, traffic accidents statistics, air travel, rail travel, rail freight, maritime transportation, inland waterways and channels, transportation research and development.',
    'Law and Crime': 'issues related to the control, prevention, and impact of crime; all law enforcement agencies, including border and customs, police, court system, prison system; terrorism, white collar crime, counterfeiting and fraud, cyber-crime, drug trafficking, domestic violence, child welfare, family law, juvenile crime.',
    'Social Welfare': 'issues related to social welfare policy, the Ministry of Social Affairs, social services, poverty assistance for low-income families and for the elderly, parental leave and child care, assistance for people with physical or mental disabilities, including early retirement pension, discounts on public services, volunteer associations (e.g., Red Cross), charities, and youth organizations.',
    'Housing': 'issues related to housing, urban affairs and community development, housing market, property tax, spatial planning, rural development, location permits, construction inspection, illegal construction, industrial and commercial building issues, national housing policy, housing for low-income individuals, rental housing, housing for the elderly, e.g., nursing homes, housing for the homeless and efforts to reduce homelessness, research related to housing.',
    'Domestic Commerce': 'issues related to banking, finance and internal commerce, including stock exchange, investments, consumer finance, mortgages, credit cards, insurance availability and cost, accounting regulation, personal, commercial, and municipal bankruptcies, programs to promote small businesses, copyrights and patents, intellectual property, natural disaster preparedness and relief, consumer safety; regulation and promotion of tourism, sports, gambling, and personal fitness; domestic commerce research.',
    'Defense': 'issues related to defense policy, military intelligence, espionage, weapons, military personnel, reserve forces, military buildings, military courts, nuclear weapons, civil defense, including firefighters and mountain rescue services, homeland security, military aid or arms sales to other countries, prisoners of war and collateral damage to civilian populations, military nuclear and hazardous waste disposal and military environmental compliance, defense alliances and agreements, direct foreign military operations, claims against military, defense research.',
    'Technology': 'issues related to science and technology transfer and international science cooperation, research policy, government space programs and space exploration, telephones and telecommunication regulation, broadcast media (television, radio, newspapers, films), weather forecasting, geological surveys, computer industry, cyber security.',
    'Foreign Trade': 'issues related to foreign trade, trade negotiations, free trade agreements, import regulation, export promotion and regulation, subsidies, private business investment and corporate development, competitiveness, exchange rates, the strength of national currency in comparison to other currencies, foreign investment and sales of companies abroad.',
    'International Affairs': 'issues related to international affairs, foreign policy and relations to other countries, issues related to the Ministry of Foreign Affairs, foreign aid, international agreements (such as Kyoto agreement on the environment, the Schengen agreement), international organizations (including United Nations, UNESCO, International Olympic Committee, International Criminal Court), NGOs, issues related to diplomacy, embassies, citizens abroad; issues related to border control; issues related to international finance, including the World Bank and International Monetary Fund, the financial situation of the EU; issues related to a foreign country that do not impact the home country; issues related to human rights in other countries, international terrorism.',
    'Government Operations': 'issues related to general government operations, the work of multiple departments, public employees, postal services, nominations and appointments, national mints, medals, and commemorative coins, management of government property, government procurement and contractors, public scandal and impeachment, claims against the government, the state inspectorate and audit, anti-corruption policies, regulation of political campaigns, political advertising and voter registration, census and statistics collection by government; issues related to local government, capital city and municipalities, including decentralization; issues related to national holidays.',
    'Public Lands': 'issues related to national parks, memorials, historic sites, and protected areas, including the management and staffing of cultural sites; museums; use of public lands and forests, establishment and management of harbors and marinas; issues related to flood control, forest fires, livestock grazing.',
    'Culture': 'issues related to cultural policies, Ministry of Culture, public spending on culture, cultural employees, issues related to support of theatres and artists; allocation of funds from the national lottery, issues related to cultural heritage.',
    'Other': 'other topics not mentioning policy agendas, including the procedures of parliamentary meetings, e.g., points of order, voting procedures, meeting logistics; interpersonal speech, e.g., greetings, personal stories, tributes, interjections, arguments between the members; rhetorical speech, e.g., jokes, literary references.'
}

# Austrian parliament-specific stop words
custom_stopwords = [
    'mr', 'mrs', 'ms', 'dr', 'madam', 'honourable', 'member', 'members', 'vp', 'sp', 'fp', 
    'minister', 'speaker', 'deputy', 'president', 'chairman', 'chair', 'schilling', 
    'secretary', 'lord', 'lady', 'question', 'order', 'point', 'debate', 'motion', 'amendment',
    'congratulations', 'congratulate', 'thanks', 'thank', 'say', 'one', 'want', 'know', 'think', 
    'believe', 'see', 'go', 'come', 'give', 'take', 'people', 'federal', 'government', 'austria', 
    'austrian', 'committee', 'call', 'said', 'already', 'please', 'request', 'proceed', 'reading',
    'course', 'welcome', 'council', 'open', 'written', 'contain', 'items', 'item', 'yes', 'no', 
    'following', 'next', 'speech', 'year', 'years', 'state', 'also', 'would', 'like', 'may', 'must', 
    'upon', 'indeed', 'session', 'meeting', 'report', 'commission', 'behalf', 'gentleman', 'gentlemen', 
    'ladies', 'applause', 'group', 'colleague', 'colleagues', 'issue', 'issues', 'chancellor', 'court', 
    'ask', 'answer', 'reply', 'regard', 'regarding', 'regards', 'respect', 'respectfully', 'sign', 
    'shall', 'procedure', 'declare', 'hear', 'minutes', 'speaking', 'close', 'abg', 'mag', 'orf', 'wait'
]

all_stopwords = list(ENGLISH_STOP_WORDS) + custom_stopwords

# Prepare segment data for topic modeling  
segment_texts = embeddings.groupby('Segment_ID')['Text'].apply(lambda x: ' '.join(x)).tolist()
segment_embeddings = np.array(embeddings.groupby('Segment_ID')['Segment_Embeddings'].first().tolist())

# Configure vectorizer
vectorizer_model = CountVectorizer(
    stop_words=all_stopwords,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    max_features=1000
)

print(f"🎯 Target categories: {len(label_list)} topics")
print(f"📊 Topic modeling data prepared:")
print(f"  • Segments for modeling: {len(segment_texts)}")
print(f"  • Embedding dimension: {segment_embeddings.shape[1]}")
print(f"📖 Topic descriptions loaded for enhanced classification")

In [ ]:
# === HIERARCHICAL BERTOPIC (RECOMMENDED APPROACH) ===
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN

def train_hierarchical_bertopic():
    """Train BERTopic with many subtopics, then map to 22 main categories."""
    print("🏗️ Training Hierarchical BERTopic with HDBSCAN...")
    
    # Configure UMAP and HDBSCAN for more granular topics
    umap_model = UMAP(n_neighbors=10, n_components=8, metric='cosine', random_state=42)
    
    # Use HDBSCAN to find clusters automatically.
    # A smaller min_cluster_size will result in more, smaller topics.
    clustering_model = HDBSCAN(
        min_cluster_size=5,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True
    )
    
    representation_model = KeyBERTInspired()
    
    topic_model_hierarchical = BERTopic(
        embedding_model="all-MiniLM-L6-v2",
        umap_model=umap_model,
        hdbscan_model=clustering_model,
        vectorizer_model=vectorizer_model,
        representation_model=representation_model,
        min_topic_size=10,  # Align with min_cluster_size
        calculate_probabilities=True,
        verbose=True
    )
    
    topics, probs = topic_model_hierarchical.fit_transform(segment_texts, embeddings=segment_embeddings)
    topic_info_hierarchical = topic_model_hierarchical.get_topic_info()
    
    print(f"✅ Hierarchical model created {len(topic_info_hierarchical[topic_info_hierarchical['Topic'] != -1])} subtopics")
    
    return topic_model_hierarchical, topics, topic_info_hierarchical

# Train hierarchical model
topic_model_hierarchical, topics_hierarchical, topic_info_hierarchical = train_hierarchical_bertopic()

In [ ]:
topic_info_hierarchical

In [ ]:
# === LLM CLASSIFICATION TO 22 CATEGORIES ===

# Load environment variables
dotenv_path = os.path.join(os.pardir, '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    print(f"✅ Loaded .env file from: {dotenv_path}")
else:
    print("⚠️ .env file not found. Ensure OPENAI_API_KEY is set in environment.")

def classify_topic_to_22_categories(topic_words, topic_id=-1):
    """Enhanced classification using detailed topic descriptions."""
    if not isinstance(topic_words, list) or not topic_words:
        return "Other"
    
    keywords_str = ', '.join(topic_words[:12])  # Use top 12 words for better context
    
    # Create detailed category descriptions for the prompt
    category_descriptions = []
    for category in label_list:
        description = majortopics_description.get(category, f"Issues related to {category.lower()}")
        category_descriptions.append(f"• {category}: {description}")
    
    categories_text = '\n'.join(category_descriptions)
    
    prompt = f"""You are analyzing topics from parliamentary debates. 

TOPIC KEYWORDS: {keywords_str}

AVAILABLE CATEGORIES WITH DESCRIPTIONS:
{categories_text}

INSTRUCTIONS:
1. Analyze the keywords carefully in the context of parliamentary discussions
2. Consider which category description best matches the semantic content of the keywords
3. Look for key thematic indicators (e.g., "economic", "health", "defense", "education", etc.)
4. If keywords relate to parliamentary procedures, interpersonal speech, or don't fit policy areas, choose "Other"
5. Choose the single best-fitting category

RESPONSE: Only output the exact category name from the list above."""

    try:
        if not os.getenv('OPENAI_API_KEY'):
            print(f"Error: OPENAI_API_KEY not set for topic {topic_id}")
            return "Other"
        
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in political science, parliamentary procedures, and policy classification. You excel at accurately mapping topic keywords to policy domains."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.05,  # Very low temperature for consistency
            max_tokens=30
        )
        
        classification = response.choices[0].message.content.strip()
        
        # Clean the response and ensure exact match
        classification = classification.replace('"', '').replace("'", "").strip()
        
        # Exact match check
        if classification in label_list:
            return classification
        
        # Fuzzy matching for partial matches
        classification_lower = classification.lower()
        for category in label_list:
            if category.lower() == classification_lower:
                return category
            # Check if classification contains the category name
            if category.lower() in classification_lower or classification_lower in category.lower():
                return category
                
        # Special handling for common variations
        category_mapping = {
            'macro': 'Macroeconomics',
            'economics': 'Macroeconomics', 
            'economic': 'Macroeeconomics',
            'rights': 'Civil Rights',
            'welfare': 'Social Welfare',
            'social': 'Social Welfare',
            'crime': 'Law and Crime',
            'legal': 'Law and Crime',
            'justice': 'Law and Crime',
            'foreign': 'International Affairs',
            'international': 'International Affairs',
            'trade': 'Foreign Trade',
            'government': 'Government Operations',
            'administration': 'Government Operations'
        }
        
        for key, mapped_category in category_mapping.items():
            if key in classification_lower:
                return mapped_category
        
        return "Other"  # Final fallback
            
    except Exception as e:
        print(f"Error classifying topic {topic_id}: {e}")
        return "Other"

def map_topics_to_22_categories(topic_info, approach_name):
    """Map topics to the 22 predefined categories using LLM classification."""
    print(f"🤖 Classifying {approach_name} topics into 22 categories...")
    
    topic_info_classified = topic_info.copy()
    topic_info_classified['Category_22'] = "Other"
    
    classification_results = []
    
    for idx, row in topic_info_classified.iterrows():
        if row['Topic'] != -1:
            topic_words = row['Representation']
            classification = classify_topic_to_22_categories(topic_words, row['Topic'])
            topic_info_classified.loc[idx, 'Category_22'] = classification
            
            classification_results.append({
                'Topic_ID': row['Topic'],
                'Keywords': ', '.join(topic_words[:5]),
                'Classification': classification,
                'Count': row['Count']
            })
    
    # Count topics per category
    category_counts = topic_info_classified[topic_info_classified['Topic'] != -1]['Category_22'].value_counts()
    
    print(f"\n📊 {approach_name} - Classification results:")
    for category in label_list:
        count = category_counts.get(category, 0)
        if count > 0:
            topic_count = len(classification_results)
            percentage = (count / topic_count * 100) if topic_count > 0 else 0
            print(f"  • {category:<20}: {count:>2} topics ({percentage:>5.1f}%)")
    
    return topic_info_classified

# Apply LLM classification
print("🚀 Applying LLM classification to map subtopics to 22 categories...")
topic_info_classified = map_topics_to_22_categories(topic_info_hierarchical, "Hierarchical BERTopic")

In [ ]:
# === CREATE FINAL 22-TOPIC MAPPING ===

def create_final_22_topic_mapping():
    """Create final mapping to 22 topics."""
    print("🏗️ Creating final 22-topic mapping...")
    
    # Create segment-level mapping
    segment_topic_map = pd.DataFrame({
        'Segment_ID': segmented_df['Segment_ID'].unique(),
        'Subtopic_ID': topics_hierarchical,
    })
    
    # Map subtopics to 22 categories
    subtopic_to_category = dict(zip(
        topic_info_classified['Topic'], 
        topic_info_classified['Category_22']
    ))
    
    segment_topic_map['Topic_22'] = segment_topic_map['Subtopic_ID'].map(subtopic_to_category)
    segment_topic_map['Topic_22'] = segment_topic_map['Topic_22'].fillna('Other')
    
    # Merge with original data
    embeddings_with_22_topics = segmented_df.merge(
        segment_topic_map, 
        on='Segment_ID', 
        how='left'
    )
    
    # Generate topic statistics
    category_stats = embeddings_with_22_topics.groupby('Topic_22').agg({
        'Segment_ID': 'nunique',
        'Text': 'count'
    }).rename(columns={
        'Segment_ID': 'Unique_Segments',
        'Text': 'Total_Speeches'
    }).sort_values('Total_Speeches', ascending=False)
    
    print(f"\n✅ Final 22-topic mapping created!")
    print(f"📊 Topic distribution:")
    for topic, stats in category_stats.iterrows():
        print(f"  • {topic:<20}: {stats['Total_Speeches']:>4} speeches, {stats['Unique_Segments']:>3} segments")
    
    return embeddings_with_22_topics, topic_info_classified, category_stats

# Create final mapping
final_embeddings, final_topic_info, category_stats = create_final_22_topic_mapping()

print(f"\n🎉 SUCCESS: Mapped all topics to 22 predefined categories!")
print(f"📈 Coverage: {len(category_stats)} out of {len(label_list)} categories have content")

In [ ]:
# === FINAL RESULTS ===

# Display final topic information
print("📋 Final Topic Classifications:")
final_display = final_topic_info[final_topic_info['Topic'] != -1][['Topic', 'Count', 'Category_22', 'Representation']].copy()
final_display['Keywords'] = final_display['Representation'].apply(lambda x: ', '.join(x[:5]))
final_display = final_display.drop('Representation', axis=1).sort_values('Count', ascending=False)

print(final_display.head(15).to_string(index=False))

# Show category coverage
print(f"\n📊 Category Coverage Summary:")
print(f"Categories with content: {len(category_stats)}/{len(label_list)}")
print(f"Most common categories:")
for category, stats in category_stats.head(10).iterrows():
    percentage = (stats['Total_Speeches'] / final_embeddings.shape[0] * 100)
    print(f"  • {category:<20}: {percentage:>5.1f}% of speeches")

# Find and display categories with no content
represented_categories = set(category_stats.index)
unrepresented_categories = set(label_list) - represented_categories

if unrepresented_categories:
    print(f"\nCategories with no content (0% of speeches):")
    for category in sorted(list(unrepresented_categories)):
        print(f"  • {category}")

In [ ]:
# === SAVE RESULTS (OPTIONAL) ===
# Uncomment to save the final results

# print("💾 Saving final results...")
# final_embeddings.to_pickle('data folder/data/AT_with_22_topics_final.pkl')
# final_topic_info.to_pickle('data folder/data/topic_info_22_categories.pkl')
# category_stats.to_pickle('data folder/data/category_statistics.pkl')
# print("✅ Results saved!")

print("\n🎉 Analysis complete! Ready to run.")